# Chapter 11. Training Deep Neural Networks

Trong chapter 10, chúng ta đã giới thiệu  vể ANN và trained our first dêp neural networks. Nhưng chúng ta chỉ đang xửu lý vài hidden layer. Vậy sẽ ra sao nếu bạn sẽtackle 1 vấn đề lớn như, hát hiện 100 đối tượng trong 1 hình ảnh có độ phân giải cao. Train a deep DNN không phải **isn't walk in the park**. Đây sẽ là 1 vài ví dụ bạn sẽ có thể đi vào dưới note book này 

* đó chính là *vanishing gradients* *exploding gradients*
* không đu dữ liệu training cho 1 mạng lưới lớn , hoặc quá đát để mình gãn label 
* trianing có thể chậm 
* Overfitting the training set, hoặc thiếu training instances hoặc chúng quá là khó hiểu. 

Note book này sẽ going tất cả các vấn đề ở rteen và chúng ta sẽ có 1 vài kỹ thuật để giải chúng. CHúng ta sẽ bắt đàu exporing với *vanishing and exploding* gradinet. Tiếp đến chúng ta sẽ đi dến *transfer learning* và *unsupervised pretraining* nó có thể hỗ trợ chúng ta tiếp cận những vấn đề lớn ngay cả khi chúng ta có ít label. Rồi chúng ta sẽ thảo luận các tối ưng để tăng lượng speed up khi train 1 với 1 mô hình lớn. Và chúng ta sẽ đi qua 1 vài những technich regularzation cho tập lớn mạng lưới neraul  

## The Vanishing/Exploding Gradients Problems

Như chúng ta đã thảo luận trong **Chapter 10**, the backpropagation algo hoạt động bằng cách đi từ output layer to input layer, lan truyền thẻ **error gradient** dọc theo dường đi.Khi thuật toán đã tính gradient của hàm mất mát liên quan tới từng tham số trong network , nó sử dụng gradient để update các parameter với a Gradient Descen step.

Thật không may, gradient thường lấy giá trị nhỏ và và nhỏ hơn khi thuật toán đi tới xuống các layer dưới. Như một kết quả, the Gradeint Descen update cập nhật các connection weight ở dưới hầy như như không thay đổi, và sẽ ko bao giờ đạt được good solution. Chúng tôi gọi là **vanishing gradients** problem. Trong 1 vài trường hợp nó có thể xảy ra, gradient có thể càng lớn và lớn hơn cho đến khi các lớp nhận được cập nhật trọng lượng cực lớn và thuật toán đó là phân kỳ. Đó là **exploding gradient**.Nói một cách tổng quát hơn, các mạng nơ-ron sâu có độ dốc không ổn định; các lớp khác nhau có thể học ở nhiều tốc độ khác nhau. Nói ngắn gọn, họ đã chỉ ra rằng với this activation và init scheme the variance của outputs của mỗi layer lớn hơn rất nhiều variance của inputs. Vào trong cái network, phương sai sẽ tiếp tục tăng sau khi mỗi layer cho đến activation function ở các lớp trên cùng. Sự saturation thực sự tồi tệ bởi thực sự nó là hàm logistic có trung bình 0.5 chứ ko 0 .Do đó,khi quá trình lan truyền khi backgrogation truyền qua nó hầu như không có  gradient truyền ngược lại the network , và những cái ít graindent tồn tại tiếp tục bị lõng như backproapgation tiến xuống các lớp trên cùn, và điều đó là thực sự k có còn gì cho các lớp dưới  

### Glorot and He Initialization
Trong bài báo của, Glorot and Bengio đã đề xuất 1 cách đáng kể vấn đề Gradient không ổn đinh. Họ đề xuất 1 cáchh giảm đáng kể vấn đề gradient không ổn đinh. Họ chỉ ra rằng chúng ta cần tín hiệu lưu thông đúng theo cả 2 hướng, theo hướng đằng trướng khi chúng ta predictions, và theo hướng ngược lại khi backprogating gradient. Chúng tôi không muốn tín hiệu bị tắt cũng như chúng tôi không muốn nó bão hoà hay bùnh nổ. Để tín hiệu đi đúng hướng các tác giả lập luận rằng chúng ta cần variance của outputs của outputs của mỗi layes bằng variance của input. và chúng ta cần gradient và chúng ta cần các gradient có phương sai bằng nhau trước và sau khi chảy qua một lớp theo hướng ngược lại. Nó thực sự không Thực sự không thể đảm bảo cả hai trừ khi lớp có số lượng đầu vào và tế bào thần kinh bằng nhau . nhưng Glorot và Bengio đã đề xuất một thỏa hiệp tốt đã được chứng minh là hoạt động rất tốt trong thực tế: trọng số kết nối của mỗi lớp phải được khởi tạo ngẫu nhiên như công thức dưới đây 

$fan_{avg} = (fan_{in} + fan_{out})/2$

Phân phối chuẩn với MAEN = 0 và 
variance $\sigma^{2}$ = $\frac{1}{fan_{avg}}$

default, keras sử dụng Glorot init với phân phối đồng nhất. khi tạo 1 layer, chúng ta có thêr thay đổi this to He init bởi setting 

```
keral_initializer="he_uniform" or 
keral_inittializer="he_normal" 

keras.layers.Dense(10,activation="relu",kernel_inititalizer="he_norlmal"
```
Nếu bạn muốn He Init với 1 uni form distribution nhưng dựa trên on $fan_{avg}$ thay vì $fan_{in}$ bạn có thể sửu dụng VarianceScaling inittialier like this:

```
he_avg_init = keras.initializers.VarianceScaling(scale=2,mode="fan_avg",
distribution='uniform'
keras.layers.Dense(10,activation="sigmoid",kernel_inittializer=he_avg_init)
```

### Nonsaturating Activation Functions 
